# Problem: Employees Whose Manager Left the Company

#### Table: Employees

| Column Name | Type     |
|-------------|----------|
| employee_id | int      |
| name        | varchar  |
| manager_id  | int      |
| salary      | int      |

In SQL, `employee_id` is the primary key for this table.
This table contains information about the employees, their salary, and the ID of their manager. Some employees do not have a manager (manager_id is null). 
 

**Find the IDs of the employees whose salary is strictly less than `$30000` and whose manager left the company. When a manager leaves the company, their information is deleted from the `Employees` table, but the reports still have their `manager_id` set to the manager that left.**

**Return the result table ordered by `employee_id`.**

**The result format is in the following example.**

### Example 1:

### Input:  

#### Employees table:

| employee_id | name      | manager_id | salary |
|-------------|-----------|------------|--------|
| 3           | Mila      | 9          | 60301  |
| 12          | Antonella | null       | 31000  |
| 13          | Emery     | null       | 67084  |
| 1           | Kalel     | 11         | 21241  |
| 9           | Mikaela   | null       | 50937  |
| 11          | Joziah    | 6          | 28485  |

### Output: 

| employee_id |
|-------------|
| 11          |

Explanation: 
The employees with a salary less than $30000 are 1 (Kalel) and 11 (Joziah).
Kalel's manager is employee 11, who is still in the company (Joziah).
Joziah's manager is employee 6, who left the company because there is no row for employee 6 as it was deleted.

In [43]:
# Import Pandas and SQLAlchemy
import pandas as pd
from sqlalchemy import create_engine, text, MetaData, Table, Column, Integer, String

In [ ]:
# Connect to SQLite database
engine = create_engine('sqlite:///example.db', echo=True)  # `echo=True` shqows SQL logs

In [47]:
# Define the metadata object
metadata = MetaData()

# Define the Employees table schema
employees = Table(
    'Employees', metadata,
    Column('employee_id', Integer, primary_key=True),
    Column('name', String),
    Column('manager_id', Integer),
    Column('salary', Integer)
)

In [48]:
# Drop table if it exists, then create the table
with engine.connect() as connection:
    # Drop the table if it exists using text() to wrap the raw SQL
    connection.execute(text("DROP TABLE IF EXISTS Employees"))
    
    # Recreate the table
    metadata.create_all(engine)  # This creates all tables defined in `metadata`

2024-12-09 12:21:02,729 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-12-09 12:21:02,732 INFO sqlalchemy.engine.Engine DROP TABLE IF EXISTS Employees
2024-12-09 12:21:02,734 INFO sqlalchemy.engine.Engine [generated in 0.00511s] ()
2024-12-09 12:21:02,811 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-12-09 12:21:02,813 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("Employees")
2024-12-09 12:21:02,816 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-12-09 12:21:02,820 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("Employees")
2024-12-09 12:21:02,821 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-12-09 12:21:02,825 INFO sqlalchemy.engine.Engine 
CREATE TABLE "Employees" (
	employee_id INTEGER NOT NULL, 
	name VARCHAR, 
	manager_id INTEGER, 
	salary INTEGER, 
	PRIMARY KEY (employee_id)
)


2024-12-09 12:21:02,826 INFO sqlalchemy.engine.Engine [no key 0.00163s] ()
2024-12-09 12:21:02,907 INFO sqlalchemy.engine.Engine COMMIT
2024-12-09 12:21:02,908 INFO sqlalchemy.e

In [49]:
# Insert data
with engine.begin() as connection:  # Automatically commits or rolls back
    connection.execute(
        employees.insert(),
        [
            {"employee_id": 3, "name": "Mila", "manager_id": 9, "salary": 60301},
            {"employee_id": 12, "name": "Antonella", "manager_id": None, "salary": 31000},
            {"employee_id": 13, "name": "Emery", "manager_id": None, "salary": 67084},
            {"employee_id": 1, "name": "Kalel", "manager_id": 11, "salary": 21241},
            {"employee_id": 9, "name": "Mikaela", "manager_id": None, "salary": 50937},
            {"employee_id": 11, "name": "Joziah", "manager_id": 6, "salary": 28485},
        ]
    )

2024-12-09 12:21:18,764 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-12-09 12:21:18,767 INFO sqlalchemy.engine.Engine INSERT INTO "Employees" (employee_id, name, manager_id, salary) VALUES (?, ?, ?, ?)
2024-12-09 12:21:18,769 INFO sqlalchemy.engine.Engine [generated in 0.00157s] [(3, 'Mila', 9, 60301), (12, 'Antonella', None, 31000), (13, 'Emery', None, 67084), (1, 'Kalel', 11, 21241), (9, 'Mikaela', None, 50937), (11, 'Joziah', 6, 28485)]
2024-12-09 12:21:18,773 INFO sqlalchemy.engine.Engine COMMIT


In [51]:
query = '''
SELECT employee_id
FROM Employees e
WHERE e.salary < 30000
  AND e.manager_id NOT IN (SELECT employee_id FROM Employees)
ORDER BY employee_id;
'''

# Using Pandas to execute and fetch results
pd.read_sql_query(query, engine)  # `engine` is the SQLAlchemy engine

2024-12-09 12:25:54,240 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-12-09 12:25:54,242 INFO sqlalchemy.engine.Engine 
SELECT employee_id
FROM Employees e
WHERE e.salary < 30000
  AND e.manager_id NOT IN (SELECT employee_id FROM Employees)
ORDER BY employee_id;

2024-12-09 12:25:54,245 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-12-09 12:25:54,251 INFO sqlalchemy.engine.Engine ROLLBACK


,employee_id
0,11
